# Exercises    
    For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like. Note that some of the questions are intentionally phrased in a vague way. It is your job to reword these as more precise questions that could be tested.

1. Has the network latency gone up since we switched internet service providers?
- null : latency performance is not related to internet providers 
- althypo : latency performance is related to internet providers 
- truepos : avg latency from one company is significantly higher than the other
- trueneg : avg latency from one company is not significantly higher than the other
- type1 : latency is better with one company statistically, but in reality it isnt
- type2 : company doesn't seem to matter with our tests, but in reality actually does
2.  Is the website redesign any good? (just as a comment, what does good even mean? For this I am assuming amt of users visiting site)
- null : website views are not related to the website change
- althypo : website views are related to website change
- truepos : website views are higher with the redesigned group compared to original
- falsepos : website views on original are higher than the redesigned
- type1 : views are higher on the new design, but in actuality it's due to just more people visiting, unrelated to the design
- type2 : our redesign doesn't change our engagement, but in reality it increases user engagement and views. 
3. Is our television ad driving more sales?
- null: advertising campaign does not influence sales
- althypo : advertising campaign influenced sales
- truepos : advertising campaing did create a larger amount of sales
- falsepos : advertising campaign did not increase sales
- type1 : sales are up, but not due to the ad. maybe its the weather
- type2 : sales aren't up, but that's due to people not needing our product. The ad is getting new customers, they just dont increase our sales enough. 

# Comparing means exercises

In [5]:
from env import host, username, password
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data

Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. Use a .05 level of significance.



- Null: time to sell a home is not different between groups
- Alternate: Time to sell a home is different

In [3]:
stats.ttest_ind_from_stats(90,15,40,100,20,50, equal_var = False)

Ttest_indResult(statistic=-2.7091418459143854, pvalue=0.00811206270346016)

#### I used ttest from stats since I had some of the descriptive stats. Negative test statistic and p value < .05 means we can reject the null hypothesis that time of sale is not significantly different. The negative test lets us know that office 2 takes longer to sell homes. 

## Load the mpg dataset and use it to answer the following questions:

### Is there a difference in fuel-efficiency in cars from 2008 vs 1999?

In [60]:
#import data
df = data('mpg')
#rename columns
df = df.rename(columns = {'cty':'city', 'hwy':'highway', 'cyl':'cylinder', 'drv':'drive'})
# create new columns for average_mileage and mileage_difference. 
df['average_mileage'] = df[['highway','city']].mean(axis = 1)
df['harmonic_mileage'] = stats.hmean(df[['city', 'highway']], axis = 1)
df['mileage_difference'] = df['highway'].sub(df['city'], axis = 0)

In [61]:
#create new data frames that filter by date
old_cars = df[df['year'] == 1999]
new_cars = df[df['year'] == 2008]
stats.levene(old_cars["average_mileage"], new_cars["average_mileage"])

LeveneResult(statistic=0.033228136671080453, pvalue=0.855517569468803)

#### The levene test determined that the variances of both old and new cars are not significantly different. 

In [62]:
## using harmonic mean
stats.levene(old_cars["harmonic_mileage"], new_cars["harmonic_mileage"])

LeveneResult(statistic=0.031071867429515194, pvalue=0.8602339200830025)

In [18]:
stats.ttest_ind(old_cars["average_mileage"], new_cars["average_mileage"])

Ttest_indResult(statistic=0.21960177245940962, pvalue=0.8263744040323578)

#### The differences in average gas mileage between the two groups is not significantly different enough to reject the null hypothesis. 

In [63]:
## using harmonic mean
stats.ttest_ind(old_cars["harmonic_mileage"], new_cars["harmonic_mileage"])

Ttest_indResult(statistic=0.3011962975077886, pvalue=0.7635345888327115)

### Are compact cars more fuel-efficient than the average car?

- Null: mpg of compact cars are not different than the population mean
- Alternative : mpg of compact cars are different than the population mean

In [64]:
# create mask to filter compact cars to compare to all the other classes of cars
compact_mask = df["class"] == "compact"
compact = df[compact_mask]
avg_car = df[compact_mask == False]

In [65]:
df["class"].unique()

array(['compact', 'midsize', 'suv', '2seater', 'minivan', 'pickup',
       'subcompact'], dtype=object)

In [67]:
stats.levene(compact["average_mileage"], avg_car["average_mileage"])

LeveneResult(statistic=11.107991264810888, pvalue=0.001000307619344077)

### Levene's test indicates that the variance between both groups are significantly different. 

In [68]:
stats.levene(compact["harmonic_mileage"], avg_car["harmonic_mileage"])

LeveneResult(statistic=7.730287593946991, pvalue=0.005875923121564377)

In [53]:
stats.ttest_ind(compact["average_mileage"], avg_car["average_mileage"], equal_var = False)

Ttest_indResult(statistic=8.128810422808078, pvalue=1.6018060656123074e-12)

In [54]:
stats.ttest_1samp(compact["average_mileage"],df["average_mileage"].mean())

Ttest_1sampResult(statistic=7.896888573132535, pvalue=4.1985637943171336e-10)

#### At first, I did this as a independent t test due to misreading the question. I went back and ran the analysis again as a 1 sample. Both have p<0.05, so we can reject our null and confirm that compact cars have significantly better gas mileage

In [69]:
## harmonic mean
stats.ttest_1samp(compact["harmonic_mileage"],df["harmonic_mileage"].mean())

Ttest_1sampResult(statistic=7.512360093161354, pvalue=1.5617666348807727e-09)

## Do manual cars get better gas mileage than automatic cars?

- Null: mpg of manual cars are not better than automatic cars
- Alternate: mpg of manual cars are better than automatic cars

In [71]:
auto = df[df['trans'].str.contains("auto")]
manual =  df[df['trans'].str.contains("manual")]

In [72]:
stats.levene(auto["average_mileage"], manual["average_mileage"])

LeveneResult(statistic=0.20075824847529639, pvalue=0.6545276355131857)

### we can confirm that the difference in variances between the two groups is insignificant and thus are equal .

In [74]:
stats.levene(auto["harmonic_mileage"], manual["harmonic_mileage"])

LeveneResult(statistic=0.013901207169738269, pvalue=0.9062462281015493)

In [76]:
stats.ttest_ind(auto["average_mileage"], manual["average_mileage"], equal_var = False)

Ttest_indResult(statistic=-4.443514012903071, pvalue=1.795224899991793e-05)

The - test value and a p<0.05 suggests that manual cars have significantly better gas mileage on average. 

In [77]:
## with harmonic mileage
stats.ttest_ind(auto["harmonic_mileage"], manual["harmonic_mileage"], equal_var = False)


Ttest_indResult(statistic=-4.47444321386703, pvalue=1.598070270207952e-05)

# reveiw notes
I had to go back and re do the anlaysis using the harmonic mean, rather than arithmetic mean at first. 

Explanation on why I had to use a harmonic mean for gas mileage can be found here : https://machinelearningmastery.com/arithmetic-geometric-and-harmonic-means-for-machine-learning/

# Correlation Exercises

In [115]:
telco_df = pd.read_csv('telco_churn.csv')

In [117]:
telco_df['avg_charges'] = telco_df['avg_charges'].str.replace('$','')

In [55]:
telco_df = pd.read_csv('query_result.csv')

In [56]:
telco_df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,1,9919-FZDED,Male,1,Yes,No,72,Yes,...,No,Yes,Yes,Yes,84.10,5981.65,No,DSL,Two year,Credit card (automatic)
7039,3,3,1,9926-PJHDQ,Female,0,Yes,Yes,72,Yes,...,No,Yes,Yes,Yes,76.80,5468.45,No,DSL,Two year,Bank transfer (automatic)
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12,No,...,Yes,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [57]:
telco_df['monthly_charges']

0       19.85
1       20.35
2       19.40
3       19.85
4       19.95
        ...  
7038    84.10
7039    76.80
7040    49.85
7041    67.85
7042    59.00
Name: monthly_charges, Length: 7043, dtype: float64

In [58]:
telco_df['tenure'] = telco_df['tenure'].astype('float')

In [59]:
stats.pearsonr(telco_df['monthly_charges'], telco_df['tenure'])

(0.24789985628615094, 4.094044991494188e-99)

In [62]:
telco_df['total_charges'] = telco_df['total_charges'].astype('float64', errors = 'ignore')

In [65]:
telco_df['TotalCharges'] = telco_df['tenure'] * telco_df['monthly_charges']

In [67]:
stats.pearsonr(telco_df['TotalCharges'], telco_df['tenure'])

(0.8265682623515945, 0.0)

In [49]:
phone_df = telco_df[telco_df['internet_service_type'] == 'None'

In [50]:
phone_df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type,TotalCharges
0,2,1,3,0030-FNXPP,Female,0,No,No,3.0,Yes,...,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check,59.55
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4.0,Yes,...,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check,81.40
2,1,1,3,0098-BOWSO,Male,0,No,No,27.0,Yes,...,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check,523.80
3,1,1,3,0107-WESLM,Male,0,No,No,1.0,Yes,...,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check,19.85
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10.0,Yes,...,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic),199.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5981,4,3,3,9950-MTGYX,Male,0,Yes,Yes,28.0,Yes,...,No internet service,No internet service,Yes,20.30,487.95,No,None,Two year,Credit card (automatic),568.40
5982,2,3,3,9953-ZMKSM,Male,0,No,No,63.0,Yes,...,No internet service,No internet service,No,25.25,1559.3,No,None,Two year,Mailed check,1590.75
5983,4,3,3,9964-WBQDJ,Female,0,Yes,No,71.0,Yes,...,No internet service,No internet service,Yes,24.40,1725.4,No,None,Two year,Credit card (automatic),1732.40
5984,3,3,3,9972-EWRJS,Female,0,Yes,Yes,67.0,Yes,...,No internet service,No internet service,Yes,19.25,1372.9,No,None,Two year,Bank transfer (automatic),1289.75


In [52]:
internet_df = telco_df[telco_df['phone_service'] == 'No']
internet_df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type,TotalCharges
2655,1,1,1,0023-HGHWL,Male,1,No,No,1.0,No,...,No,No,Yes,25.10,25.1,Yes,DSL,Month-to-month,Electronic check,25.10
2656,3,1,1,0032-PGELS,Female,0,Yes,Yes,1.0,No,...,No,No,No,30.50,30.5,Yes,DSL,Month-to-month,Bank transfer (automatic),30.50
2658,2,1,1,0076-LVEPS,Male,0,No,Yes,29.0,No,...,No,No,Yes,45.00,1242.45,No,DSL,Month-to-month,Mailed check,1305.00
2663,3,1,1,0117-LFRMW,Male,0,Yes,Yes,37.0,No,...,No,No,No,40.20,1448.8,Yes,DSL,Month-to-month,Bank transfer (automatic),1487.40
2664,2,1,1,0128-MKWSG,Female,0,No,Yes,26.0,No,...,No,Yes,No,45.80,1147,No,DSL,Month-to-month,Mailed check,1190.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7035,4,3,1,9885-MFVSU,Female,0,Yes,Yes,35.0,No,...,Yes,No,Yes,46.35,1662.05,No,DSL,Two year,Credit card (automatic),1622.25
7036,3,3,1,9888-ZCUMM,Male,0,Yes,Yes,64.0,No,...,Yes,Yes,Yes,60.05,3845.45,No,DSL,Two year,Bank transfer (automatic),3843.20
7037,3,3,1,9906-NHHVC,Female,1,No,No,65.0,No,...,Yes,Yes,Yes,53.50,3517.9,No,DSL,Two year,Bank transfer (automatic),3477.50
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12.0,No,...,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check,598.20


In [57]:
have_phones = telco_df[telco_df['phone_service'] == 'Yes']

In [60]:
both_df = have_phones[have_phones['internet_service_type'] != 'None']

In [62]:
stats.pearsonr(phone_df['TotalCharges'], phone_df['tenure'])

(0.9843702784237233, 0.0)

In [63]:
stats.pearsonr(internet_df['TotalCharges'], internet_df['tenure'])

(0.9542624310656165, 0.0)

In [64]:
stats.pearsonr(both_df['TotalCharges'], both_df['tenure'])

(0.9586810298135677, 0.0)

In [4]:
def get_emp_url(username, password, host):
    db_url = f'mysql+pymysql://{username}:{password}@{host}/employees'
    return db_url

In [5]:
url = get_emp_url(username, password, host)

In [12]:
emp_df = pd.read_sql("""SELECT emp_no, hire_date, title, titles.from_date as 'title_from', 
                               titles.to_date as 'title_to', salary, salaries.from_date as 'salary_from', 
                               salaries.to_date as 'salary_to'  
                        FROM employees
                        join titles using (emp_no)
                        join salaries using (emp_no);""", url)

In [35]:
emp_df.head(1)

,emp_no,hire_date,title,title_from,title_to,salary,salary_from,salary_to
0,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,60117,1986-06-26,1987-06-26


In [42]:
emp_df['salary_to'] = emp_df['salary_to'].astype('str').str.replace('9999-01-01','2021-05-11')

In [44]:
emp_df.head(50)

,emp_no,hire_date,title,title_from,title_to,salary,salary_from,salary_to
0,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,60117,1986-06-26,1987-06-26
1,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,62102,1987-06-26,1988-06-25
2,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,66074,1988-06-25,1989-06-25
3,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,66596,1989-06-25,1990-06-25
4,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,66961,1990-06-25,1991-06-25
5,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,71046,1991-06-25,1992-06-24
6,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,74333,1992-06-24,1993-06-24
7,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,75286,1993-06-24,1994-06-24
8,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,75994,1994-06-24,1995-06-24
9,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,76884,1995-06-24,1996-06-23


In [45]:
emp_df['tenure'] = pd.to_datetime(emp_df['salary_to'])-pd.to_datetime(emp_df['hire_date']) 

In [48]:
emp_df['tenure'] = emp_df['tenure'].dt.days.astype('int')

In [51]:
stats.pearsonr(emp_df['salary'], emp_df['tenure'])

(0.33263632021818157, 0.0)

In [68]:
countref = emp_df.groupby('emp_no').count()[['title']].rename(columns = {'title': 'titlecount'})

In [70]:
merged_df = pd.merge(emp_df, countref, how= 'left', on = 'emp_no')

In [71]:
merged_df

,emp_no,hire_date,title,title_from,title_to,salary,salary_from,salary_to,tenure,titlecount
0,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,60117,1986-06-26,1987-06-26,365,17
1,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,62102,1987-06-26,1988-06-25,730,17
2,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,66074,1988-06-25,1989-06-25,1095,17
3,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,66596,1989-06-25,1990-06-25,1460,17
4,10001,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,66961,1990-06-25,1991-06-25,1825,17
...,...,...,...,...,...,...,...,...,...,...
4638502,499999,1997-11-30,Engineer,1997-11-30,9999-01-01,63707,1997-11-30,1998-11-30,365,5
4638503,499999,1997-11-30,Engineer,1997-11-30,9999-01-01,67043,1998-11-30,1999-11-30,730,5
4638504,499999,1997-11-30,Engineer,1997-11-30,9999-01-01,70745,1999-11-30,2000-11-29,1095,5
4638505,499999,1997-11-30,Engineer,1997-11-30,9999-01-01,74327,2000-11-29,2001-11-29,1460,5


In [73]:
stats.pearsonr(merged_df['tenure'], merged_df['titlecount'])

(0.0081749596681162, 2.1822663663200402e-69)

In [52]:
sleep_df = data('sleepstudy')

In [53]:
sleep_df

,Reaction,Days,Subject
1,249.5600,0,308
2,258.7047,1,308
3,250.8006,2,308
4,321.4398,3,308
5,356.8519,4,308
...,...,...,...
176,329.6076,5,372
177,334.4818,6,372
178,343.2199,7,372
179,369.1417,8,372


In [27]:
stats.pearsonr(sleep_df['Reaction'], sleep_df['Days'])

(0.5352302262650253, 9.894096322214812e-15)

In [2]:
subs = {}
for subject in sleep_df['Subject'].unique():
    subs[subject] = {}
    subs[subject]['Days'] = sleep_df['Days'][sleep_df['Subject'] == subject]
    subs[subject]['Reaction'] = sleep_df['Reaction'][sleep_df['Subject'] == subject]

NameError: name 'sleep_df' is not defined

In [77]:
subs.keys()

dict_keys([308, 309, 310, 330, 331, 332, 333, 334, 335, 337, 349, 350, 351, 352, 369, 370, 371, 372])

In [1]:
for subject in subs:
    r, p = stats.pearsonr(subs[subject]['Days'], subs[subject]['Reaction'])
    print(f'Pearsons r is {r:.3f} with a p_value of {p: .3f}')

NameError: name 'subs' is not defined

# Chi squared exercises

1. Use the following contingency table to help answer the question of whether using a macbook and being a codeup student are independent of each other.

In [7]:
chi2, p, degf, expected = stats.chi2_contingency([[49 , 20] , [1 , 30]])
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

chi^2 = 36.6526
p     = 0.0000


2. Choose another 2 categorical variables from the mpg dataset and perform a chi2 contingency table test with them. Be sure to state your null and alternative hypotheses.

Hypotheses 
- Null: Transmission type is not correlated with the model year
- Alternate: Transmission is correlated with model year

In [8]:
mpg = data('mpg')

In [9]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


In [10]:
mpg['trans'] = mpg.trans.str[:-4]

In [11]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto,f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual,f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual,f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto,f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto,f,16,26,p,compact


In [18]:
observed = pd.crosstab(mpg.year, mpg.trans)

In [20]:
observed

trans,auto,manual
year,,
1999,74,43
2008,83,34


In [23]:
chi2, p, degf, expected = stats.chi2_contingency(observed)
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

chi^2 = 1.2388
p     = 0.2657


3. Use the data from the employees database to answer these questions:
- Is an employee's gender independent of whether an employee works in sales or marketing? (only look at current employees)
- Is an employee's gender independent of whether or not they are or have been a manager?

In [24]:
def get_emp_url(username, password, host):
    db_url = f'mysql+pymysql://{username}:{password}@{host}/employees'
    return db_url
url = get_emp_url(username, password, host)

In [32]:
emp_df = pd.read_sql('''SELECT emp_no, first_name, last_name, dept_no, gender, to_date
                        FROM employees
                        JOIN dept_emp USING(emp_no)
                        WHERE dept_no IN ('d007','d001') 
                        AND to_date LIKE '%9999%';''', url)

In [33]:
emp_df

,emp_no,first_name,last_name,dept_no,gender,to_date
0,10002,Bezalel,Simmel,d007,F,9999-01-01
1,10016,Kazuhito,Cappelletti,d007,M,9999-01-01
2,10017,Cristinel,Bouloucos,d001,F,9999-01-01
3,10041,Uri,Lenart,d007,F,9999-01-01
4,10050,Yinghua,Dredge,d007,M,9999-01-01
...,...,...,...,...,...,...
52538,499976,Guozhong,Felder,d007,M,9999-01-01
52539,499980,Gino,Usery,d007,M,9999-01-01
52540,499986,Nathan,Ranta,d007,F,9999-01-01
52541,499987,Rimli,Dusink,d007,F,9999-01-01


In [34]:
observed1 = pd.crosstab(emp_df.dept_no, emp_df.gender)

In [36]:
observed1

gender,F,M
dept_no,,
d001,5864,8978
d007,14999,22702


In [37]:
chi2, p, degf, expected = stats.chi2_contingency(observed1)
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

chi^2 = 0.3240
p     = 0.5692


In [68]:
managers = pd.read_sql('''SELECT emp_no, first_name, last_name, dept_no, gender, to_date
                          FROM employees
                          LEFT JOIN dept_manager USING(emp_no);''', url)

In [65]:
non_man = pd.read_sql('''SELECT emp_no, first_name, last_name, dept_no, gender, to_date
                         FROM employees
                         LEFT JOIN dept_manager USING(emp_no)
                         WHERE dept_no IS NULL;''', url)

In [81]:
managers

,emp_no,first_name,last_name,dept_no,gender,to_date
0,10001,Georgi,Facello,None,M,None
1,10002,Bezalel,Simmel,None,F,None
2,10003,Parto,Bamford,None,M,None
3,10004,Chirstian,Koblick,None,M,None
4,10005,Kyoichi,Maliniak,None,M,None
...,...,...,...,...,...,...
300019,499995,Dekang,Lichtner,None,F,None
300020,499996,Zito,Baaz,None,M,None
300021,499997,Berhard,Lenart,None,M,None
300022,499998,Patricia,Breugel,None,M,None


In [82]:
is_manager = managers.dept_no.notnull()

In [83]:
is_manager

0         False
1         False
2         False
3         False
4         False
          ...  
300019    False
300020    False
300021    False
300022    False
300023    False
Name: dept_no, Length: 300024, dtype: bool

In [88]:
observed3= pd.crosstab(managers.gender, is_manager)

In [91]:
chi2, p, degf, expected = stats.chi2_contingency(observed3)
print(f'chi^2 = {chi2:.4f}')
print(f'p     = {p:.4f}')

chi^2 = 1.4567
p     = 0.2275
